#### Search Engine with Tools and Agents 

In [1]:
## Arxiv-Research
## Tools-Creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [2]:
## Used the inbuilt tool of wikipedia

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]

In [5]:
## Custom Tools

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

c:\Users\gargs\Downloads\LangchainProjects\3-Search Engine\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/administration-overview")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
documents = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorstores = FAISS.from_documents(documents, embeddings)
retriever = vectorstores.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BAE4CED010>, search_kwargs={})

In [7]:
from langchain_core.tools import Tool

retriever_tool = Tool(
    name="Langsmith-search",
    description="Search any information about Langsmith",
    func=lambda q: retriever.invoke(q),
)

retriever_tool.name


'Langsmith-search'

In [8]:
tools = [wiki, arxiv, retriever_tool]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\gargs\\Downloads\\LangchainProjects\\3-Search Engine\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langsmith-search', description='Search any information about Langsmith', func=<function <lambda> at 0x000001BAD1921620>)]

In [10]:
## Runs all tools with Agents and LLM models
## Tools, LLM -> AgentExecutor

from dotenv import load_dotenv
load_dotenv()
import os
import groq

api_key = os.getenv("GROQ_API_KEY")
groq.api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model_name = "openai/gpt-oss-120b", groq_api_key = api_key)

In [13]:
## Prompt Template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI assistant. "
    "You may use tools when helpful. "
    "Reason step by step, then provide the final answer."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI assistant. You may use tools when helpful. Reason step by step, then provide the final answer.'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]